In [18]:
import pandas as pd
import datetime
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [40]:
df = pd.read_csv('New_dataset_Account_with_datetime.csv')

In [23]:
data = pd.read_csv('New_dataset_clean.csv')

In [27]:
len(set(df['Third Party Name']))

84

In [14]:
df.head()

,Account No,Balance,Amount,Third Party Account No,Third Party Name,Datetime
0,101531259.0,124.00,-859.00,NaN,LBG,2023-01-01 00:00:00
1,101531259.0,116.01,-7.99,NaN,Disney,2023-01-01 00:00:00
2,101531259.0,16.01,-100.00,NaN,Grand Union BJJ,2023-01-01 00:00:00
3,101531259.0,-63.99,-80.00,NaN,Matalan,2023-01-01 09:37:00
4,101531259.0,-247.99,-184.00,NaN,Matalan,2023-01-01 12:00:00


In [41]:
df_cleaned = df.dropna(subset=['Third Party Name'])
df_cleaned = df_cleaned[df_cleaned['Amount'] <= 0]
df_cleaned = df_cleaned.drop(columns=['Third Party Account No'])
df_cleaned['Third Party Name'] = df_cleaned['Third Party Name'].str.lower()

In [35]:
third_party = np.unique(df_cleaned['Third Party Name'])
len(third_party)

73

In [36]:
third_party

array(['a cut above', 'a yarn story', 'amazon', 'barbiee boutique',
       "blackwell's", 'blizzard', 'boots', 'brilliant brushes',
       'cass art', 'cex', 'coffee #1', 'collector cave', 'coop local',
       'costa coffee', 'craftastic', 'deliveroo', 'disney', 'etsy',
       'fat face', 'fitted stitch', 'five senses art', 'foyles',
       'frankie & bennies', 'gamestation', 'gap kids', 'grand union bjj',
       'halifax', 'happy days home', 'head', 'hmv', 'hobby lobby',
       'hobbycraft', 'jd sports', 'justeat', 'kew house', 'kings arms',
       'lavender primary', 'lbg', 'lloyds pharmacy', 'loosely fitted',
       'mamas & papas', 'matalan', 'millets', 'mojang studios',
       'mothercare', 'mountain warehouse', 'netflix', 'north face',
       'pets corner', 'puregym', 'reebok', 'remedy plus care', 'revella',
       'rose & crown', 'rugbyfields', 'sainsbury', 'sainsbury local',
       'selfridges', 'specsavers', 'sports direct', 'squareonix',
       'starbucks', 'stitch by stitch'

In [48]:
def add_label(to_account):

    if re.search(r"cut", to_account):
        return "haircut"
    
    if re.search(r"amazon", to_account):
        return "online-shopping"
        
    if re.search(r"pet", to_account):
        return "pet"
    
    if re.search(r"selfridges", to_account):
        return "supermarket"
    
    if re.search(r"kid|mama|mother|primary|barbiee", to_account):
        return "children"
    
    if re.search(r"deliveroo|eat", to_account):
        return "takeaway"
    
    if re.search(r"etsy", to_account):
        return "gift"
    
    if re.search(r"blackwell|foyles|works", to_account):
        return "book"
    
    if re.search(r"boots|remedy|pharmacy|hospital", to_account):
        return "healthcare"  
    
    if re.search(r"blizzard|cex|game|mojang|squareonix|xbox", to_account):
        return "game"
    
    if re.search(r"brushes|art|craft|hobby", to_account):
        return "art"
    
    if re.search(r"coffee|starbucks", to_account):
        return "coffee"
    
    if re.search(r"collector|hmv", to_account):
        return "video_and_music"
    
    if re.search(r"coop|sainsbury|tesco", to_account):
        return "convenient_shop"
       
    if re.search(r"disney|netflix", to_account):
        return "streaming"  
    
    if re.search(r"frankie", to_account):
        return "restaurant"  
    
    if re.search(r"bjj|head|mountain|gym|field", to_account):
        return "sport"
    
    if re.search(r"halifax|lbg", to_account):
        return "bank"
    
    if re.search(r"home|house", to_account):
        return "home"
    
    if re.search(r"crown|arms", to_account):
        return "bar"
    
    if re.search(r"specsaver|vision", to_account):
        return "glasses"
    
    if re.search(r"park|high", to_account):
        return "park"
    
    return "clothes"
    
classification_df = df_cleaned.copy()
classification_df['category'] = classification_df['Third Party Name'].apply(add_label) 

In [50]:
classification_df.head()

,Account No,Balance,Amount,Third Party Name,Datetime,category
0,101531259.0,124.00,-859.00,lbg,2023-01-01 00:00:00,bank
1,101531259.0,116.01,-7.99,disney,2023-01-01 00:00:00,streaming
2,101531259.0,16.01,-100.00,grand union bjj,2023-01-01 00:00:00,sport
3,101531259.0,-63.99,-80.00,matalan,2023-01-01 09:37:00,clothes
4,101531259.0,-247.99,-184.00,matalan,2023-01-01 12:00:00,clothes


In [51]:
classification_df.to_csv("new_dataset_with_customer_behavior.csv", index=False)

In [59]:
classification_df['Datetime'] = pd.to_datetime(classification_df['Datetime'])
classification_df['month'] = classification_df['Datetime'].dt.month
monthly_behaviors_expenses = classification_df.groupby(['Account No', 'month', 'category'])['Amount'].sum().reset_index()
monthly_behaviors_expenses = monthly_behaviors_expenses.pivot_table(index=['Account No','month'], columns='category', values='Amount',fill_value=0, aggfunc='first').reset_index()
monthly_behaviors_expenses['total'] = monthly_behaviors_expenses.drop(['Account No','month'], axis=1).sum(axis=1)
monthly_behaviors_expenses

category,Account No,month,art,bank,bar,book,children,clothes,coffee,convenient_shop,...,online-shopping,park,pet,restaurant,sport,streaming,supermarket,takeaway,video_and_music,total
0,101531259.0,1,-24.98,-859,0.0,0.00,0,-794.50,0.00,-118.89,...,0.0,0,0,0,-409.99,-15.98,0.0,-17.69,0.0,-2241.03
1,101531259.0,2,-1.50,-859,0.0,0.00,0,0.00,0.00,-254.88,...,0.0,0,0,0,-100.00,-7.99,0.0,0.00,0.0,-1223.37
2,101531259.0,3,-24.98,-859,0.0,0.00,0,0.00,0.00,-107.05,...,0.0,0,0,0,-131.99,-7.99,0.0,-15.00,0.0,-1146.01
3,101531259.0,4,-8.49,-859,0.0,0.00,0,0.00,0.00,-299.51,...,0.0,0,0,0,-206.99,-7.99,0.0,-28.63,0.0,-1410.61
4,101531259.0,5,-109.94,-859,0.0,0.00,0,0.00,0.00,-240.15,...,0.0,0,0,0,-151.98,-7.99,0.0,0.00,0.0,-1369.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11596,999752672.0,8,0.00,-652,0.0,-28.96,0,-485.46,-48.40,-476.74,...,0.0,0,0,0,0.00,-15.99,0.0,-58.44,0.0,-1765.99
11597,999752672.0,9,0.00,-652,0.0,-29.95,0,-585.97,-66.70,-599.06,...,0.0,0,0,0,0.00,-15.99,0.0,0.00,0.0,-1949.67
11598,999752672.0,10,0.00,-652,0.0,-4.99,0,-286.45,-50.95,-449.25,...,0.0,0,0,0,0.00,-15.99,0.0,0.00,0.0,-1459.63
11599,999752672.0,11,0.00,-652,0.0,-16.97,0,-158.95,-80.90,-556.27,...,0.0,0,0,0,0.00,-15.99,0.0,-34.85,0.0,-1515.93


In [62]:
# percentage of customer behavior spendings
def calculate_ratio(data, category):
    for cat in category:
        data['ratio of ' + cat] =(data[cat]  / data['total']) * 100 
    return data

category = np.unique(classification_df['category'])
monthly_behaviors_expenses = calculate_ratio(monthly_behaviors_expenses, category)

In [64]:
monthly_behaviors_expenses.to_csv("new_dataset_monthly_behaviors_expenses.csv", index=False)

In [66]:
classification_df['Datetime'] = pd.to_datetime(classification_df['Datetime'])
classification_df['year'] = classification_df['Datetime'].dt.year
yearly_behaviors_expenses = classification_df.groupby(['Account No', 'year', 'category'])['Amount'].sum().reset_index()
yearly_behaviors_expenses = yearly_behaviors_expenses.pivot_table(index=['Account No','year'], columns='category', values='Amount',fill_value=0, aggfunc='first').reset_index()
yearly_behaviors_expenses['total'] = yearly_behaviors_expenses.drop(['Account No','year'], axis=1).sum(axis=1)

yearly_behaviors_expenses = calculate_ratio(yearly_behaviors_expenses, category)

yearly_behaviors_expenses

category,Account No,year,art,bank,bar,book,children,clothes,coffee,convenient_shop,...,ratio of home,ratio of online-shopping,ratio of park,ratio of pet,ratio of restaurant,ratio of sport,ratio of streaming,ratio of supermarket,ratio of takeaway,ratio of video_and_music
0,101531259.0,2023,-474.69,-10308,0.0,0.00,0,-2356.99,0.0,-2394.86,...,-0.0,-0.000000,-0.0,-0.0,-0.0,10.834669,0.543920,-0.000000,0.495076,-0.0
1,104832000.0,2023,0.00,-10332,0.0,0.00,0,0.00,0.0,-3619.33,...,-0.0,-0.000000,-0.0,-0.0,-0.0,7.888310,1.123714,-0.000000,1.237620,-0.0
2,105375973.0,2023,0.00,0,-12.9,-462.18,0,-6030.61,0.0,-258.79,...,-0.0,1.492657,-0.0,-0.0,-0.0,2.263571,1.905977,23.978771,1.337402,-0.0
3,106601471.0,2023,0.00,0,-23.9,-348.63,0,-4162.03,0.0,-1039.83,...,-0.0,5.071127,-0.0,-0.0,-0.0,-0.000000,2.266621,0.434536,5.140200,-0.0
4,108481285.0,2023,-425.87,0,0.0,-889.94,0,-6370.74,0.0,-1528.08,...,-0.0,7.277498,-0.0,-0.0,-0.0,1.952318,0.752981,2.022227,0.550706,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,995615876.0,2023,0.00,-7644,-26.0,0.00,0,-7452.27,0.0,-1433.67,...,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.000000,-0.000000,-0.000000,2.177535,-0.0
972,996042490.0,2023,-874.86,-6012,0.0,-89.88,0,-4439.38,0.0,-2887.46,...,-0.0,0.537534,-0.0,-0.0,-0.0,1.366350,1.150497,0.385058,1.761541,-0.0
973,998390769.0,2023,0.00,-12156,0.0,0.00,0,-3091.43,0.0,-3866.23,...,-0.0,0.028091,-0.0,-0.0,-0.0,-0.000000,1.349507,-0.000000,0.693278,-0.0
974,998405607.0,2023,-14.97,0,0.0,-69.91,0,-4106.90,0.0,-2431.58,...,-0.0,-0.000000,-0.0,-0.0,-0.0,15.061886,2.176458,-0.000000,1.829595,-0.0


In [67]:
yearly_behaviors_expenses.to_csv("new_dataset_yearly_behaviors_expenses.csv", index=False)